In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
X_train_trf1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/X_train_trf1_v3.csv')
X_val_trf1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/X_val_trf1_v3.csv')
X_test_trf1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/X_test_trf1_v3.csv')

y_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/y_train_nn_v3.csv').to_numpy().flatten()
y_val= pd.read_csv('/content/drive/My Drive/Colab Notebooks/y_val_nn_v3.csv').to_numpy().flatten()
y_test= pd.read_csv('/content/drive/My Drive/Colab Notebooks/y_test_nn_v3.csv').to_numpy().flatten()

movielist = pd.concat([X_train_trf1.loc[:,['primaryTitle', 'movieId_ori','movieId']],
           X_val_trf1.loc[:,['primaryTitle', 'movieId_ori','movieId']],
           X_test_trf1.loc[:,['primaryTitle', 'movieId_ori','movieId']]
], axis = 0).drop_duplicates()

userlist = pd.concat([X_train_trf1.loc[:,['userId', 'userId_ori']],
           X_val_trf1.loc[:,['userId', 'userId_ori']],
           X_test_trf1.loc[:,['userId', 'userId_ori']]
], axis = 0).drop_duplicates()

In [3]:
userid_movie = X_test_trf1.loc[:,['userId_ori','movieId_ori']]
movie_user_rating = pd.concat(
    [userid_movie,
         pd.Series(y_test, name ='score')], axis = 1
)
movie_user_rating

,userId_ori,movieId_ori,score
0,89464,104374,3.5
1,1920,82459,4.0
2,159816,5617,4.0
3,30643,104841,4.0
4,74429,1562,1.0
...,...,...,...
68428,111038,1077,3.0
68429,2177,4010,3.0
68430,103611,1343,4.5
68431,94154,112552,4.0


In [4]:
# Convert 9bins to 2bins func
# index 0 to 6: movie scores given by user, where 0:0.5, 1:1, 2:1.5, 3:2, 4:2.5, 5: 3,  
def nineToTwoBins(y_dataset):
    y_dataset = (y_dataset >= 3.5).astype(int)
    return y_dataset

# Convert to binary bins, 0 if score < 3.5, 1 if score >= 3.5:
y_train = (nineToTwoBins(y_train))
y_val = (nineToTwoBins(y_val))
y_test = (nineToTwoBins(y_test))


# Get Predictions (Probability) for validation



In [5]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import DenseFeatures

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.layers import Dropout

best_model_bo = keras.models.load_model('/content/drive/My Drive/Colab Notebooks/best_model_bo.h5')

for i in [X_val_trf1, X_test_trf1]:
  i.drop(['rating_month__1',
 'rating_month__2',
 'rating_month__3',
 'rating_month__4',
 'rating_month__5',
 'rating_month__6',
 'rating_month__7',
 'rating_month__8',
 'rating_month__9',
 'rating_month__10',
 'rating_month__11',
 'rating_month__12',
 'rating_year',
 'primaryTitle',
 'movieId_ori',
 'userId_ori'], axis=1, inplace = True)

preds = best_model_bo.predict(X_test_trf1,batch_size = 400)

In [6]:
movie_user_proba = pd.concat(
    [userid_movie,
         pd.Series(preds.flatten(), name ='probability')], axis = 1
)

# Hit Rate

# Get all the unique users in val set

1) Function

-> take in userID from validation set, filter by 1 particular ID

-> get list of movies & coresponding actual ratings that is rated by this user in val set


-> feed in X_val (also need to filter by userID) get the predicted probability of each of the movies in this validation set, 

concat this probability with the movieID, then sort by probability

-> conca



In [7]:
movie_user_rating.userId_ori.unique()

array([ 89464,   1920, 159816,  30643,  74429,  61869,  39905, 111038,
        61379,  34217, 118906,   9620,   2177,  36883,  29054, 147898,
        65365,  32855,  49004,  58402, 160747,  80446,  81896,  32064,
        20179,  90691,  92046,  37438, 152107,  18306,  81403,  60305,
        57548,  47628,  18995,   2982, 113791,  53837, 143049,  99547,
        14831,  25554,  46638,  41608,  30024, 135672, 126180, 109480,
        32778,  62737, 148789,  54353,  22175,  39669, 144475, 140838,
        22253, 115102,  51238, 122011,  44147,  93451, 145626,  41487,
        29021,  61803, 146221,  17474,  34987, 109090,  71403,  31567,
       131293,  32462,  96495,  29803,  17783,  81216,  73300, 150005,
       107650, 151214,   1748,  62199, 116447,  83426, 140862,  30879,
        80974,  97452, 122803, 106507, 162516,  26672, 162047, 103611,
       103631,  62524,  75830, 141938, 105747,  98227,  96818,  29749,
        31776,  93855,  73949,  52190, 110971, 120793,  61010,  85859,
      

In [8]:
unique_user_ids = movie_user_rating.userId_ori.unique()

def sorted_rating_by_user(userID_ori, movie_user_rating_df):
  movie_rating = movie_user_rating_df.loc[movie_user_rating_df['userId_ori'] == userID_ori,
                                          ['movieId_ori','score']].sort_values(by ='score',ascending=False
                                          )
  return movie_rating                                  

def sorted_proba_by_user(userID_ori, movie_user_proba_df):
  movie_rating = movie_user_proba_df.loc[movie_user_proba_df['userId_ori'] == userID_ori,
                                          ['movieId_ori','probability']].sort_values(by ='probability',ascending=False
                                          )
  return movie_rating                                  
  

In [9]:
def hit_ratio(rating_df, probability_df, k):
  movie_array_true = rating_df.movieId_ori.to_numpy().flatten()[0:k]
  movie_array_false = probability_df.movieId_ori.to_numpy().flatten()[0:k]
  numerator = len(np.intersect1d(movie_array_true,movie_array_false))
  ratio = numerator/k
  return numerator/k

hit_ratio(sorted_rating_by_user(30643, movie_user_rating), sorted_proba_by_user(30643, movie_user_proba), k=100)

0.24

In [11]:
# Get average hit_ratio across all users in validation set. Change k only.
def get_average_hit_ratio(k, userid_list, movie_user_rating_df, movie_user_proba_df):
  all_hit_ratios = []
  for i in userid_list:
    rating = sorted_rating_by_user(i, movie_user_rating_df)
    proba = sorted_proba_by_user(i, movie_user_proba_df)
    hr = hit_ratio(rating, proba, k)
    all_hit_ratios.append(hr)
  return np.mean(np.array(all_hit_ratios))

get_average_hit_ratio(50, unique_user_ids, movie_user_rating, movie_user_proba)

0.16797872340425532

In [12]:
for k in [1,5,10,20,50,100]:
  average_hr = get_average_hit_ratio(k, unique_user_ids, movie_user_rating, movie_user_proba)
  print('Average Hit Ratio for n = {}:  {}'.format(k, average_hr))

Average Hit Ratio for n = 1:  0.0
Average Hit Ratio for n = 5:  0.030851063829787237
Average Hit Ratio for n = 10:  0.05585106382978723
Average Hit Ratio for n = 20:  0.09441489361702128
Average Hit Ratio for n = 50:  0.16797872340425532
Average Hit Ratio for n = 100:  0.2832978723404255


# Computation of NDCG

In [13]:
# Get Cumulative Gain of a given user in validation set. Change k only.
def cumulative_gain(rating_df, probability_df, k):
  # rating_df: Given User's Dataframe of MovieId, & score with MovieId sorted by score 
  movie_array_true = rating_df.movieId_ori.to_numpy().flatten()[0:k]
  # probability_df: Given User's Dataframe of MovieId, & probability with MovieId sorted by probability
  movie_array_false = probability_df.movieId_ori.to_numpy().flatten()[0:k]
  cg= len(np.intersect1d(movie_array_true,movie_array_false))
  return cg

cumulative_gain(sorted_rating_by_user(18628, movie_user_rating), sorted_proba_by_user(18628, movie_user_proba), 10)

0

In [14]:
#test code for computation of DCG, IGNORE
common_ele = np.intersect1d(np.array([1,2,3,4,5,6,7,8,9,10]),
                        np.array([1,12,2,14,15,11,13,16,17,18])
                        ) # set of common movieIds in both arrays
indices = np.where(np.in1d(np.array([1,12,2,14,15,11,13,16,17,18]), common_ele))[0]+ 1
np.sum(1/np.log2(indices + 1))

1.5

In [15]:
# Get Discounted Cumulative Gain of a given user in validation set. Change k only.
def dis_cumulative_gain(rating_df, probability_df, k):

  # rating_df: Given User's Dataframe of MovieId, & score with MovieId sorted by score 
  movie_array_true = rating_df.movieId_ori.to_numpy().flatten()[0:k]
  ##print('ideal_movie: ', movie_array_true)

  # probability_df: Given User's Dataframe of MovieId, & probability with MovieId sorted by probability
  movie_array_false = probability_df.movieId_ori.to_numpy().flatten()[0:k]
  ##print('predicted_movie:', movie_array_false)

  # set of common movieIds in both arrays
  common_elements = np.intersect1d(movie_array_true,
                        movie_array_false)
  ##print('common_movie: ', common_elements)

  # find indices of common elements of movie_array_true & movie_array_false
  # inside movie_array_false
  # add 1 to change index 0 to 1, 1 to 2,...
  indices = np.where(np.in1d(movie_array_false, common_elements))[0]+ 1
  ##print('indices of common_movie: ', indices)
  dcg = np.sum(1/np.log2(indices + 1))
  #print('dcg:', dcg)
  return dcg


In [16]:
# Get Ideal DCG for user
# Get Discounted Cumulative Gain of a given user in validation set. Change k only.
def ideal_cumulative_gain(rating_df, k):

  # rating_df: Given User's Dataframe of MovieId, & score with MovieId sorted by score 
  movie_array_true = rating_df.movieId_ori.to_numpy().flatten()[0:k]
  ##print('ideal_movie: ', movie_array_true)

  # find indices of ideal scenario
  # add 1 to change index 0 to 1, 1 to 2,...
  indices = np.array([indice for indice in range(len(movie_array_true))]
  ) + 1
  ##print(indices)
  ideal_dcg = np.sum(1/np.log2(indices + 1))
  ##print(ideal_dcg)
  return ideal_dcg


In [17]:
#NDCG per user
def user_NDCG(rating_df, probability_df, k):
  dcg_ = dis_cumulative_gain(rating_df, probability_df, k)
  idcg_ = ideal_cumulative_gain(rating_df, k)
  ndcg_ = dcg_ / idcg_
  return(ndcg_)

user_NDCG(sorted_rating_by_user(85844, movie_user_rating), sorted_proba_by_user(85844, movie_user_proba), 10)

0.37466610587647853

In [18]:
# get Average NDCG across all users, change k only
def get_average_ndcg(k, userid_list, movie_user_rating_df, movie_user_proba_df):
  all_ndcg = []
  for i in userid_list:
    rating = sorted_rating_by_user(i, movie_user_rating_df)
    proba = sorted_proba_by_user(i, movie_user_proba_df)
    user_ndcg = user_NDCG(rating, proba, k)
    all_ndcg.append(user_ndcg)
  return np.mean(np.array(all_ndcg))

get_average_ndcg(50, unique_user_ids, movie_user_rating, movie_user_proba)

0.19722457712283922

In [19]:
for k in [1,5,10,20,50,100]:
  average_ndcg = get_average_ndcg(k, unique_user_ids, movie_user_rating, movie_user_proba)
  print('Average Hit Ratio for n = {}:  {}'.format(k, average_ndcg))

Average Hit Ratio for n = 1:  0.0
Average Hit Ratio for n = 5:  0.031392295775200973
Average Hit Ratio for n = 10:  0.06184369478431828
Average Hit Ratio for n = 20:  0.1100109469917045
Average Hit Ratio for n = 50:  0.19722457712283922
Average Hit Ratio for n = 100:  0.3339062048459825
